In [1]:
# Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import tensorflow as tf
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.


# Read Data

In [2]:
real = pd.read_csv("True.csv")
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
fake = pd.read_csv("Fake.csv")
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
# Create Target based on Real and Fake data
real['Category'] = 1
fake['Category'] = 0

In [5]:
print(real.shape)
real.head()

(21417, 5)


,title,text,subject,date,Category
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
print(fake.shape)
fake.head()

(23481, 5)


,title,text,subject,date,Category
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [7]:
dataset = pd.concat([real, fake]).reset_index(drop=True)

In [8]:
print(dataset.shape)
dataset.head()

(44898, 5)


,title,text,subject,date,Category
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


# Data Analysis

In [9]:
dataset['final_text'] = dataset['title'] + dataset['text']
dataset['final_text'].head()

0    As U.S. budget fight looms, Republicans flip t...
1    U.S. military to accept transgender recruits o...
2    Senior U.S. Republican senator: 'Let Mr. Muell...
3    FBI Russia probe helped by Australian diplomat...
4    Trump wants Postal Service to charge 'much mor...
Name: final_text, dtype: object

In [10]:
dataset['Category'].value_counts()

0    23481
1    21417
Name: Category, dtype: int64

In [11]:
dataset[['Category','subject','final_text']].groupby(['Category','subject']).count()

final_text
Category subject                    
0        Government News        1570
         Middle-east             778
         News                   9050
         US_News                 783
         left-news              4459
         politics               6841
1        politicsNews          11272
         worldnews             10145

# Data Cleaning

#### Preprocessing Text to get Stemmed and Lemmatized Corpus

In [12]:
porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [13]:
stemmed_text = []
lemmatized_text = []
final_text_result = []
for text in dataset['final_text']:
    result = re.sub('[^a-zA-Z]', ' ', text)
    result = result.lower()
    result = result.split()
    result = [r for r in result if r not in set(stopwords.words('english'))]
    final_text_result.append(" ".join(result))
    stemmed_result = [porter_stemmer.stem(r) for r in result]
    stemmed_text.append(" ".join(stemmed_result))
    lemmatized_result = [lemmatizer.lemmatize(r) for r in result]
    lemmatized_text.append(" ".join(lemmatized_result))

In [14]:
print(len(final_text_result))
print(len(stemmed_text))
print(len(lemmatized_text))

44898
44898
44898


# Create models using CountVectorizer and TFIDF Vectorizer

In [15]:
def get_prediction(vectorizer, classifier, X_train, X_test, y_train, y_test):
    pipe = Pipeline([('vector', vectorizer),
                    ('model', classifier)])
    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuarcy: {}".format(round(accuracy_score(y_test, y_pred)*100,2)))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix: \n", cm)
    print("Classification Report: \n", classification_report(y_test, y_pred))

In [16]:
print("******USING STEMMED TEXT********")
X_train, X_test, y_train, y_test = train_test_split(stemmed_text, dataset['Category'], test_size = 0.3, random_state= 0)
classifiers = [LogisticRegression(), SGDClassifier(), MultinomialNB(), BernoulliNB(), LinearSVC(),
              KNeighborsClassifier(n_neighbors=5), DecisionTreeClassifier(), GradientBoostingClassifier(), 
               RandomForestClassifier(), XGBClassifier()]
for classifier in classifiers:
    print("\n\n", classifier)
    print("***********Usng Count Vectorizer****************")
    get_prediction(CountVectorizer(), classifier, X_train, X_test, y_train, y_test)
    print("***********Usng TFIDF Vectorizer****************")
    get_prediction(TfidfVectorizer(), classifier, X_train, X_test, y_train, y_test)

******USING STEMMED TEXT********


 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
***********Usng Count Vectorizer****************


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuarcy: 99.6
Confusion Matrix: 
 [[7043   26]
 [  28 6373]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7069
           1       1.00      1.00      1.00      6401

   micro avg       1.00      1.00      1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 98.66
Confusion Matrix: 
 [[6963  106]
 [  74 6327]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.98      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_i

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Accuarcy: 99.29
Confusion Matrix: 
 [[7010   59]
 [  37 6364]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 99.09
Confusion Matrix: 
 [[6998   71]
 [  51 6350]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
***********Usng Count Vectorizer****************
Accuarcy: 

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuarcy: 99.52
Confusion Matrix: 
 [[7035   34]
 [  31 6370]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7069
           1       0.99      1.00      0.99      6401

   micro avg       1.00      1.00      1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 99.47
Confusion Matrix: 
 [[7031   38]
 [  33 6368]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuarcy: 96.07
Confusion Matrix: 
 [[6935  134]
 [ 396 6005]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.95      0.98      0.96      7069
           1       0.98      0.94      0.96      6401

   micro avg       0.96      0.96      0.96     13470
   macro avg       0.96      0.96      0.96     13470
weighted avg       0.96      0.96      0.96     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 96.32
Confusion Matrix: 
 [[6949  120]
 [ 376 6025]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      7069
           1       0.98      0.94      0.96      6401

   micro avg       0.96      0.96      0.96     13470
   macro avg       0.96      0.96      0.96     13470
weighted avg       0.96      0.96      0.96     13470



 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate

In [17]:
print("******USING LEMMATIZED TEXT********")
X_train, X_test, y_train, y_test = train_test_split(lemmatized_text, dataset['Category'], test_size = 0.3, random_state= 0)
classifiers = [LogisticRegression(), SGDClassifier(), MultinomialNB(), BernoulliNB(), LinearSVC(),
              KNeighborsClassifier(n_neighbors=5), DecisionTreeClassifier(), GradientBoostingClassifier(), 
               RandomForestClassifier(), XGBClassifier()]
for classifier in classifiers:
    print("\n\n", classifier)
    print("***********Usng Count Vectorizer****************")
    get_prediction(CountVectorizer(), classifier, X_train, X_test, y_train, y_test)
    print("***********Usng TFIDF Vectorizer****************")
    get_prediction(TfidfVectorizer(), classifier, X_train, X_test, y_train, y_test)

******USING LEMMATIZED TEXT********


 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
***********Usng Count Vectorizer****************


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuarcy: 99.64
Confusion Matrix: 
 [[7046   23]
 [  26 6375]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7069
           1       1.00      1.00      1.00      6401

   micro avg       1.00      1.00      1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 98.73
Confusion Matrix: 
 [[6966  103]
 [  68 6333]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.98      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Accuarcy: 99.5
Confusion Matrix: 
 [[7026   43]
 [  24 6377]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      7069
           1       0.99      1.00      0.99      6401

   micro avg       1.00      1.00      1.00     13470
   macro avg       0.99      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 99.14
Confusion Matrix: 
 [[6996   73]
 [  43 6358]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
***********Usng Count Vectorizer****************
Accuarcy: 9

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuarcy: 99.6
Confusion Matrix: 
 [[7043   26]
 [  28 6373]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7069
           1       1.00      1.00      1.00      6401

   micro avg       1.00      1.00      1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 99.53
Confusion Matrix: 
 [[7034   35]
 [  28 6373]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7069
           1       0.99      1.00      1.00      6401

   micro avg       1.00      1.00      1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuarcy: 97.08
Confusion Matrix: 
 [[6970   99]
 [ 294 6107]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      7069
           1       0.98      0.95      0.97      6401

   micro avg       0.97      0.97      0.97     13470
   macro avg       0.97      0.97      0.97     13470
weighted avg       0.97      0.97      0.97     13470

***********Usng TFIDF Vectorizer****************
Accuarcy: 98.82
Confusion Matrix: 
 [[7031   38]
 [ 121 6280]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      7069
           1       0.99      0.98      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate

##### Here we see that the Decision Tree is giving best with lemmatized texts with accuracy score of 99.71%

# Using LSTM

In [18]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [19]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping

#### Using One hot representation using Stemmed Text

In [20]:
voc_size = 5000
onehot_stemmed_text = [one_hot(word, voc_size) for word in stemmed_text]
print(len(onehot_stemmed_text))
onehot_stemmed_text[0]

44898


[4451,
 2178,
 1171,
 2430,
 3898,
 3285,
 2055,
 3843,
 1332,
 3379,
 3776,
 3898,
 55,
 4451,
 2473,
 4118,
 4578,
 1131,
 1701,
 2748,
 508,
 4491,
 2358,
 4432,
 4317,
 2055,
 3776,
 1769,
 1479,
 2178,
 3607,
 3502,
 4580,
 7,
 2004,
 2276,
 3898,
 4451,
 1759,
 2265,
 532,
 2351,
 49,
 1965,
 2748,
 3525,
 1056,
 3773,
 3338,
 4122,
 3119,
 3448,
 501,
 2421,
 4985,
 3570,
 2452,
 3119,
 2521,
 1984,
 2808,
 3338,
 2178,
 1171,
 1906,
 3201,
 3832,
 3925,
 1382,
 3628,
 3617,
 3836,
 1760,
 3568,
 460,
 3898,
 3448,
 3502,
 1827,
 2473,
 1282,
 3010,
 3,
 3898,
 3686,
 2160,
 2178,
 1781,
 3059,
 4122,
 2085,
 758,
 3686,
 3862,
 1781,
 1258,
 4082,
 2156,
 4122,
 3242,
 1414,
 220,
 4152,
 1677,
 4242,
 1987,
 3087,
 606,
 3854,
 3,
 75,
 2514,
 507,
 1082,
 3857,
 1781,
 1258,
 4082,
 2156,
 4122,
 776,
 532,
 1942,
 4121,
 4800,
 3438,
 3195,
 2475,
 3884,
 3242,
 2085,
 1082,
 121,
 276,
 1719,
 596,
 4491,
 714,
 776,
 2055,
 3776,
 125,
 2081,
 3447,
 522,
 2234,
 1688,
 38

In [21]:
sent_length = 400
embedded_text = pad_sequences(onehot_stemmed_text, padding='pre', maxlen=sent_length)
print(embedded_text)

[[3773 3338 4122 ... 4979 4118 2586]
 [   0    0    0 ...  658  749  591]
 [   0    0    0 ... 4100 4400 3884]
 ...
 [ 107 2513 2382 ... 3427   69  772]
 [   0    0    0 ... 3427 4777 3162]
 [4585 3359 1641 ... 3427 1881  772]]


In [22]:
embedding_vector_features = 600
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model1.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 600)          3000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               280400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 3,280,501
Trainable params: 3,280,501
Non-trainable params: 0
_________________________________________________________________


In [23]:
X_final = np.array(embedded_text)
y_final = dataset['Category']

In [24]:
X_final.shape,y_final.shape

((44898, 400), (44898,))

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 0)

In [26]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, mode='auto')

In [27]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size = 256, callbacks=([reduce_lr, early_stop]))

Instructions for updating:
Use tf.cast instead.
Train on 31428 samples, validate on 13470 samples
Epoch 1/20
31428/31428 [==============================] - 399s 13ms/step - loss: 0.1807 - acc: 0.9365 - val_loss: 0.0675 - val_acc: 0.9788
Epoch 2/20
31428/31428 [==============================] - 397s 13ms/step - loss: 0.0572 - acc: 0.9820 - val_loss: 0.0915 - val_acc: 0.9638
Epoch 3/20
31428/31428 [==============================] - 395s 13ms/step - loss: 0.0512 - acc: 0.9838 - val_loss: 0.0595 - val_acc: 0.9826
Epoch 4/20
31428/31428 [==============================] - 396s 13ms/step - loss: 0.1248 - acc: 0.9639 - val_loss: 0.1457 - val_acc: 0.9366
Epoch 5/20
31428/31428 [==============================] - 396s 13ms/step - loss: 0.0764 - acc: 0.9727 - val_loss: 0.0891 - val_acc: 0.9712
Epoch 6/20
31428/31428 [==============================] - 396s 13ms/step - loss: 0.0582 - acc: 0.9804 - val_loss: 0.0523 - val_acc: 0.9839
Epoch 7/20
31428/31428 [==============================] - 405s 13ms/

#### Evaluate model

In [28]:
y_pred = model1.predict_classes(X_test)
print("Accuarcy: {}".format(round(accuracy_score(y_test, y_pred)*100,2)))

Accuarcy: 99.17


In [29]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)

Confusion Matrix: 
 [[7025   44]
 [  68 6333]]


In [30]:
print("Classification Report: \n", classification_report(y_test, y_pred))

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



#### Using One hot representation using Lemmatized Text

In [31]:
voc_size = 5000
onehot_lemmatized_text = [one_hot(word, voc_size) for word in lemmatized_text]
print(len(onehot_lemmatized_text))
onehot_lemmatized_text[0]

44898


[4451,
 2178,
 1171,
 2430,
 3898,
 3285,
 2055,
 3843,
 2169,
 3379,
 2206,
 3898,
 55,
 4451,
 2473,
 1556,
 4578,
 1131,
 52,
 4507,
 508,
 4491,
 2358,
 4432,
 1260,
 2055,
 2206,
 1769,
 3235,
 2178,
 3607,
 1279,
 4580,
 7,
 2004,
 2276,
 3898,
 4451,
 1111,
 2265,
 532,
 742,
 1626,
 1965,
 2748,
 3525,
 1056,
 3773,
 1222,
 855,
 4709,
 1311,
 2257,
 1577,
 4985,
 3570,
 2452,
 4709,
 2521,
 857,
 3911,
 1222,
 2178,
 1171,
 2806,
 4356,
 147,
 4611,
 2701,
 3628,
 4594,
 825,
 3745,
 3568,
 460,
 3898,
 3448,
 3502,
 1827,
 2473,
 3957,
 3010,
 3,
 3898,
 3686,
 2160,
 2178,
 936,
 496,
 855,
 2085,
 758,
 3686,
 843,
 936,
 1258,
 190,
 803,
 855,
 3242,
 1414,
 4180,
 2084,
 1677,
 275,
 1987,
 3087,
 4127,
 976,
 3,
 3302,
 3930,
 4843,
 1082,
 2861,
 936,
 1258,
 190,
 803,
 855,
 776,
 532,
 1942,
 4121,
 664,
 2361,
 3195,
 4869,
 3884,
 3242,
 2085,
 4206,
 121,
 276,
 1719,
 3254,
 4491,
 4616,
 776,
 2055,
 2206,
 125,
 4965,
 1448,
 522,
 786,
 1688,
 3884,
 532,
 22

In [32]:
sent_length = 400
embedded_text = pad_sequences(onehot_lemmatized_text, padding='pre', maxlen=sent_length)
print(embedded_text)

[[3773 1222  855 ... 4979 1556 2586]
 [   0    0    0 ... 4359  749 1032]
 [   0    0    0 ... 4451 1400 3884]
 ...
 [ 107 2513 2382 ... 3427   69  772]
 [   0    0    0 ... 3427 4777 3162]
 [3575 3359 1641 ... 3427 1881  772]]


In [33]:
embedding_vector_features = 600
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 600)          3000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               280400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 3,280,501
Trainable params: 3,280,501
Non-trainable params: 0
_________________________________________________________________


In [34]:
X_final = np.array(embedded_text)
y_final = dataset['Category']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 0)

In [36]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size = 256, callbacks=([reduce_lr, early_stop]))

Train on 31428 samples, validate on 13470 samples
Epoch 1/20
31428/31428 [==============================] - 395s 13ms/step - loss: 0.1704 - acc: 0.9383 - val_loss: 0.0702 - val_acc: 0.9763
Epoch 2/20
31428/31428 [==============================] - 393s 12ms/step - loss: 0.1042 - acc: 0.9597 - val_loss: 0.0752 - val_acc: 0.9773
Epoch 3/20
31428/31428 [==============================] - 393s 13ms/step - loss: 0.0578 - acc: 0.9813 - val_loss: 0.0567 - val_acc: 0.9828
Epoch 4/20
31428/31428 [==============================] - 392s 12ms/step - loss: 0.0355 - acc: 0.9886 - val_loss: 0.1203 - val_acc: 0.9554
Epoch 5/20
31428/31428 [==============================] - 402s 13ms/step - loss: 0.0374 - acc: 0.9875 - val_loss: 0.0454 - val_acc: 0.9860
Epoch 6/20
31428/31428 [==============================] - 391s 12ms/step - loss: 0.0160 - acc: 0.9956 - val_loss: 0.0425 - val_acc: 0.9863
Epoch 7/20
31428/31428 [==============================] - 392s 12ms/step - loss: 0.0139 - acc: 0.9962 - val_loss: 0.

#### Evaluate model

In [37]:
y_pred = model2.predict_classes(X_test)
print("Accuarcy: {}".format(round(accuracy_score(y_test, y_pred)*100,2)))

Accuarcy: 98.93


In [38]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)

Confusion Matrix: 
 [[7010   59]
 [  85 6316]]


In [39]:
print("Classification Report: \n", classification_report(y_test, y_pred))

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

   micro avg       0.99      0.99      0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [40]:
X_final = np.array(final_text_result)
y_final = dataset['Category']